### Please install the required Python modules/SDKs

In [ ]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

# Form Recognizer Model Export Demo

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.exceptions import ResourceNotFoundError

## Set up source Form Recognizer object

In [ ]:
AZURE_FORM_RECOGNIZER_ENDPOINT = "ENTER ENDPOINT HERE"
AZURE_FORM_RECOGNIZER_KEY = "ENTER KEY HERE"

In [ ]:
endpoint = AZURE_FORM_RECOGNIZER_ENDPOINT
key = AZURE_FORM_RECOGNIZER_KEY

In [ ]:
source_form_recognizer_client = FormTrainingClient(endpoint=endpoint, credential=AzureKeyCredential(key))

### IF you will have any of the models in your resources, it will be listed here. 

In [ ]:
account_properties = source_form_recognizer_client.get_account_properties()
print("Current Model Counts: {}".format(account_properties.custom_model_count))
print("Maximum Model Limit: {}".format(account_properties.custom_model_limit))
print("Available Model Limit: {}".format(account_properties.custom_model_limit - account_properties.custom_model_count))

In [ ]:
custom_form_recognizer_models = source_form_recognizer_client.list_custom_models()
print("Listing all the form recognizer models -----")
[print(model.model_id) for model in custom_form_recognizer_models]

## Selecting the model (to be copied to another Form Recognizer resource)

In [ ]:
# Select a model from the list 
model_id = 'ENTER MODEL HERE'

In [ ]:
selected_form_recognizer_model = source_form_recognizer_client.get_custom_model(model_id=model_id)

In [ ]:
selected_form_recognizer_model.to_dict()

## Setting target resource (of the same type)

In [ ]:
TARGET_FORM_RECOGNIZER_ENDPOINT = "ENTER ENDPOINT HERE"
TARGET_FORM_RECOGNIZER_KEY = "ENTER KEY HERE"
TARGET_REGION = "ENTER REGION HERE"
TARGET_RESOURCE_ID = "ENTER RESOURCE ID HERE"

In [ ]:
target_form_recognizer_client = FormTrainingClient(endpoint=TARGET_FORM_RECOGNIZER_ENDPOINT, credential=AzureKeyCredential(TARGET_FORM_RECOGNIZER_KEY))

In [ ]:
target = target_form_recognizer_client.get_copy_authorization(
    resource_region=TARGET_REGION,
    resource_id=TARGET_RESOURCE_ID
)
# model ID that target client will use to access the model once copy is complete
print("Model ID: {}".format(target["modelId"]))
# [END get_copy_authorization]


In [ ]:
poller = source_form_recognizer_client.begin_copy_model(
    model_id=selected_form_recognizer_model.model_id,
    target=target  # output from target client's call to get_copy_authorization()
)
copied_over_model = poller.result()

print("Model ID: {}".format(copied_over_model.model_id))
print("Status: {}".format(copied_over_model.status))

In [ ]:
# Verify that the target Form Recognizer resource's model count has increased by 1

target_account_properties = target_form_recognizer_client.get_account_properties()
print("Current Model Counts: {}".format(target_account_properties.custom_model_count))
print("Maximum Model Limit: {}".format(target_account_properties.custom_model_limit))
print("Available Model Limit: {}".format(target_account_properties.custom_model_limit - target_account_properties.custom_model_count))
